In [1]:
import pandas as pd
import os
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore') #supresses warnings for now
#warnings.filterwarnings(action='once') #shows warnings once

dataFilePath = os.path.join(os.getcwd(),'Data')

kicsSVdf = pd.read_csv(os.path.join(dataFilePath,'kics_structural_variations.txt'), sep = '\t', header = 0)
#kicsTEdf = pd.read_csv(os.path.join(dataFilePath,'kics_transposable_elements.txt'), sep = '\t', header = 0)
lfsSVdf = pd.read_csv(os.path.join(dataFilePath,'lfs_structural_variations.txt'), sep = '\t', header = 0)
#lfsTEdf = pd.read_csv(os.path.join(dataFilePath,'lfs_transposable_elements.txt'), sep = '\t', header = 0)
kicsClinicdf = pd.read_csv(os.path.join(dataFilePath,'kics_clinical_main.tsv'), sep = '\t', header = 0)
lfsClinicdf = pd.read_csv(os.path.join(dataFilePath,'lfs_clinical_main.tsv'), sep = '\t', header = 0)


In [2]:
"""

filter dfs

"""
def maxGDAFFilter(colName: str, df: pd.DataFrame)->pd.DataFrame:
    temp = df[colName].str.contains(pat=',')
    Index = list(np.where(temp == True)[0])
    temp = df[colName].iloc[Index]
    tempIndex = temp.index

    for i in tempIndex:
        tempList = temp[i].split(',')
        tempList = [float(a) for a in tempList]
        maxTemp = max(tempList)
        df[colName][i] = maxTemp

    df[colName] = df[colName].astype('float64')
    
    return df
    
kicsSVdf = maxGDAFFilter('GD_AF', kicsSVdf)
lfsSVdf = maxGDAFFilter('GD_AF', lfsSVdf)

In [3]:
listValSearch = ['291775', '298313', '315666', '320118', '18_1907']
# searched for kID of 63, 83, 156, 171, 219
# kID 120 141 and 232 do not have any matches...
#what would make this that much better is if it was a dict. Computationally less heavy?

for i in listValSearch: 
    kicsSVdf.drop(kicsSVdf[kicsSVdf['sample_id'].str.contains(i)==True].index, inplace = True)

In [26]:
# """
# Create a new column for each of the df abbv id to join on that column

# """

# kicsSVdf['sample_id'] = kicsSVdf['sample_id'].astype(str)
# kicsClinicdf['CCP germline'] = kicsssClinicdf['CCP germline'].astype(str)

# kicsSVdf['sample_id'] = kicsSVdf['sample_id'].str.replace(" ", "")
# kicsClinicdf['CCP germline'] = kicsClinicdf['CCP germline'].str.replace(" ", "")

# kicsClinicdf['abbv_id'] = kicsClinicdf['CCP germline'].map(clinDic)
# kicsSVdf['abbv_id'] = kicsSVdf['sample_id'].map(SVDic)

# #print(kicsSVdf['abbv_id'].value_counts())
# #print(kicsClinicdf['abbv_id'].value_counts())

# bigKicsDf = pd.merge(kicsSVdf[['sample_id','SV type','abbv_id']], 
#                      kicsClinicdf[['KiCS ID', 'CCP germline', 'tumour_class', 'abbv_id']],
#                      left_on = 'abbv_id', 
#                      right_on = 'abbv_id', 
#                      how='outer', 
#                      indicator = True)

# #print(len(SVDic))
# #print(bigKicsDf['_merge'].value_counts())
# #print(len(kicsSVdf['sample_id'].unique())) 191

# bigKicsDf = pd.merge(kicsSVdf[['sample_id','SV type','abbv_id']], 
#                      kicsClinicdf[['KiCS ID', 'CCP germline', 'tumour_class', 'abbv_id']],
#                      left_on = 'abbv_id', 
#                      right_on = 'abbv_id', 
#                      how='inner')
# #print(bigKicsDf['abbv_id'].unique()) #total 145, dropped 46 samples


13898     42995
318953    35900
13887     35345
317052    34095
274027    33675
          ...  
2697      16559
315666    16395
320118    16384
298313    16384
271113    15693
Name: abbv_id, Length: 191, dtype: int64
286900    4
301002    4
286720    4
284386    4
8559      3
         ..
316456    1
318953    1
318092    1
318128    1
1851      1
Name: abbv_id, Length: 309, dtype: int64
both          5018943
left_only     1257743
right_only        197
Name: _merge, dtype: int64
['5255' '313703' '312528' '332546' '1605' '320118' '315585' '319104'
 '271117' '4156' '7027' '291775' '286952' '287846' '317981' '286720'
 '328003' '5639' '296263' '321967' '315487' '331387' '315488' '316071'
 '303763' '317052' '313785' '323895' '298111' '308437' '271116' '308436'
 '291647' '301002' '297377' '330754' '1508' '273308' '305082' '328067'
 '294368' '318953' '293107' '307735' '304458' '298313' '4155' '300683'
 '294212' '284385' '318128' '301286' '284387' '1619' '313706' '312306'
 '271113' '301784' '33

In [50]:
"""
Do the same for LFS
"""

def kIdAbbv(clinicDf, svDf, clinicCol, svCol):
    
    clinicDf.drop(clinicDf[clinicDf[clinicCol]=='Not applicable'].index, inplace = True)
    
    svDf[svCol] = svDf[svCol].astype(str)
    clinicDf[clinicCol] = clinicDf[clinicCol].astype(str)
    svDf[svCol] =  svDf[svCol].str.replace(" ", "")
    clinicDf[clinicCol] = clinicDf[clinicCol].str.replace(" ", "")
    
    list1 = svDf[svCol].unique()
    list2 = clinicDf[clinicCol].unique()
    
    svDict = dict()
    clinicDict = dict()
    
    for i in range (len(list1)):
        tempList = re.findall('\d{3,}', list1[i])

        #if empty, take the second one
        if not tempList:
            tempList = re.findall('\d+', list1[i])
            svDict[list1[i]] = tempList[1]

        #if not empty 
        else:
            svDict[list1[i]] = tempList[0]


    for i in range (len(list2)):    
        tempList = re.findall('\d{3,}', list2[i])
    
        #if empty, take the second one
        if not tempList:
            tempList = re.findall('\d+', list2[i])
            clinicDict[list2[i]] = tempList[1]

        #if not empty 
        else:
            clinicDict[list2[i]] = tempList[0]
    

    clinicDf['abbv_id'] = clinicDf[clinicCol].map(clinicDict)
    svDf['abbv_id'] = svDf[svCol].map(svDict)
    
    print(len(svDf[svCol].unique()))
    
    df = pd.merge(svDf[[svCol,'SV type','abbv_id']], 
                     clinicDf[[clinicCol, 'tumour_class', 'abbv_id']],
                     left_on = 'abbv_id', 
                     right_on = 'abbv_id', 
                     how='inner')

    return df

In [51]:
bigDf = kIdAbbv(kicsClinicdf, kicsSVdf, 'CCP germline', 'sample_id')
print(len(bigDf['abbv_id'].unique()))

191
145


In [52]:
print(bigDf)


        sample_id SV type abbv_id     CCP germline       tumour_class
0         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS
1         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS
2         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS
3         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS
4         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS
...           ...     ...     ...              ...                ...
5018938    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA
5018939    292600     DUP  292600           292600  LEUKEMIA/LYMPHOMA
5018940    292600     DUP  292600           292600  LEUKEMIA/LYMPHOMA
5018941    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA
5018942    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA

[5018943 rows x 5 columns]


In [38]:
print(len(lfsSVdf['sample_id'].unique())) #only 85 to begin with D:
print(lfsSVdf['sample_id'].unique())

85
[4257 '4257' 'SJACT008_G' '808' 808 2085 1087 1092 3805 '3805' '3136A'
 '4033' 4033 2447 353 3806 4854 1478 2262 4028 2957 3688 93 1357 2565 4329
 '4329' '4535_S1' '3298A' '2349' 2349 2446 3473 5567 3273 3885 3432 1843
 3763 3503 '3503' 'PD13489b' '2463' 2463 '2760A' '3643' 3643 'SJACT001_G'
 '4227' 4227 5537 3367 '3367' 'SJACT002_G' '2815' 2815 5536 3634 1476
 '1476' 'SJACT005_G' '3425' 3425 3365 2413 '2413' 'SJACT007_G' '3356' 3356
 3361 '3361' '3687_2' '3332' 3332 94 1355 2784 2465 3319 '3319' 'PD7195b'
 '3334' 3334 352 2765]


In [37]:
print(lfsClinicdf.columns)

"""
drop unaffected? and NA? 
Which column do i use for id
"""



Index(['sample', 'meth_id', 'p53', 'tm_donor', 'tissue_type',
       'cancer_diagnosis', 'active_cancer', 'gender', 'cancer_atdraw',
       'cancer_atdraw_y5', 'cancer_atdraw_y1', 'firstcancer_atdraw',
       'age_diff_closest_cancer', 'systemic_treatment_atdraw', 'ageofonset',
       'agesamplecollection', 'cancer_num', 'cancer1_age_diff', 'cancer1',
       'cancer1_ageofonset', 'cancer2', 'cancer2_ageofonset', 'cancer3',
       'cancer3_ageofonset', 'cancer4', 'cancer4_ageofonset', 'cancer5',
       'cancer5_ageofonset', 'cancer6', 'cancer6_ageofonset', 'cancer7',
       'cancer7_ageofonset', 'cancer8', 'cancer8_ageofonset', 'protein',
       'family', 'mdm2', 'gdna.exon.intron', 'gdna.transcript', 'gdna.start',
       'gdna.end', 'gdna.genomic.notation', 'gdna.change', 'cdna.base.change',
       'cdna.codon', 'cdna.transcript', 'cdna.codon.notation',
       'protein.codon.change', 'protein.codon.num', 'Variant_Classification',
       'Variant_Type', 'alleles', 'MRN', 'EXT#', 'WGS', 

In [ ]:
"""
Apply on TE
Apply on Filtered Prelim Graphs
"""

